In [ ]:
import re
from datasets import load_dataset
from pathlib import Path

seed = 42

## change the path depending on your system
hub_basepath = Path(r"C:\Users\samba\.cache\huggingface\hub")

## Sentiment Analysis and Question/Answering
Sentiment Analysis from [FPB](https://huggingface.co/datasets/AdaptLLM/FPB)</br>
Question/Answering from [Headline](https://huggingface.co/datasets/AdaptLLM/Headline)</br>

In [3]:
## load_dataset raises an error after download due to tab seperated dataset.
try:
    load_dataset("AdaptLLM/FPB")
except: ...
try:
    load_dataset("AdaptLLM/Headline")
except: ...

Generating train split: 0 examples [00:00, ? examples/s]

Failed to read file 'C:\Users\samba\.cache\huggingface\hub\datasets--AdaptLLM--FPB\snapshots\7f203bd82f0b2b01ce391b9451c642dd732cf381\train.csv' with error <class 'pandas.errors.ParserError'>: Error tokenizing data. C error: Expected 3 fields in line 11, saw 5



README.md:   0%|          | 0.00/6.62k [00:00<?, ?B/s]

c:\Users\samba\OneDrive - Durham University\L4\Project\code\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samba\.cache\huggingface\hub\datasets--AdaptLLM--Headline. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train.csv:   0%|          | 0.00/9.87M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Failed to read file 'C:\Users\samba\.cache\huggingface\hub\datasets--AdaptLLM--Headline\snapshots\68cf1056f3ed51d39b945d004259473759555559\train.csv' with error <class 'pandas.errors.ParserError'>: Error tokenizing data. C error: Expected 1 fields in line 10, saw 4



## Topic Classification
From huggingface [Sujet-Finance-Instruct-177k](https://huggingface.co/datasets/sujet-ai/Sujet-Finance-Instruct-177k)

In [ ]:
dataset_id = "sujet-ai/Sujet-Finance-Instruct-177k"

topics = ['Analyst Update', 'Fed | Central Banks', 'Company | Product News', 'Treasuries | Corporate Debt', 'Dividend', 'Earnings', 'Energy | Oil', 'Financials', 'Currencies', 'General News | Opinion', 'Gold | Metals | Materials', 'IPO', 'Legal | Regulation', 'M&A | Investments', 'Macro', 'Markets', 'Politics', 'Personnel Change', 'Stock Commentary', 'Stock Movement']
topics_label2id = {label: i for i, label in enumerate(topics)}

def topic_class_filter(example):
    return example["task_type"] == "topic_classification"

def topic_mapping(example):
    example["answer"] = topics_label2id[example["answer"]]
    return example

def text_cleaning(example):
    example["user_prompt"] = re.sub(r"https://t.co/.+", "", example["user_prompt"]).strip(" ")
    return example

rmv_cols = ["Unnamed: 0", "inputs", "system_prompt", "task_type", "dataset", "index_level", "conversation_id"]
rnm_cols = {"answer": "label", "user_prompt": "text"}

dataset = (load_dataset(dataset_id, split="train")
            .filter(topic_class_filter)
            .map(topic_mapping)
            .map(text_cleaning)
            .remove_columns(rmv_cols)
            .rename_columns(rnm_cols))
dataset = dataset.train_test_split(test_size=0.05, seed=seed)
dataset

Map:   0%|          | 0/16990 [00:00<?, ? examples/s]

Map:   0%|          | 0/16990 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 16140
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 850
    })
})

In [ ]:
dataset["train"].to_csv(hub_basepath / "datasets--Sujet--TopicClassification" / "train.csv", header=False)
dataset["test"].to_csv(hub_basepath / "datasets--Sujet--TopicClassification" / "test.csv", header=False)

Creating CSV from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

88000